# Import thư viện

### Nạp thư viện

In [ ]:
!python -m pip install --upgrade pip

! pip -q install --upgrade pip
! pip -q install -r req.txt
! pip install -U accelerate
! pip install -U transformers

### Import thư viện

In [91]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

# Tiền xử lí dữ liệu



### Nạp model

In [125]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels=5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### lưu model (optional)

In [101]:
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\bpe.codes',
 'tokenizer\\added_tokens.json')

### Nạp dataset

In [131]:
# Tải dữ liệu từ file CSV
dataset = load_dataset("csv", data_files={"train": "dataset.csv", "test": "dataset.csv"})

# Lấy danh sách các nhãn gốc từ cột 'intent'
intent_labels = dataset["train"]["intent"]  # Cột 'intent' chứa các nhãn gốc

# Khởi tạo LabelEncoder để mã hóa nhãn thành chỉ số
label_encoder = LabelEncoder()

# Mã hóa các nhãn gốc thành các chỉ số
label_encoder.fit(intent_labels)  # Fit trên toàn bộ nhãn từ cột 'intent'

# In ra các nhãn gốc và các chỉ số đã mã hóa
print("Danh sách nhãn gốc:", label_encoder.classes_)  # In mảng nhãn gốc
print("Các nhãn đã mã hóa:", label_encoder.transform(intent_labels))  # In nhãn đã mã hóa thành chỉ số

# Mã hóa nhãn intent thành chỉ số và tokenize văn bản
def encode_labels_and_tokenize(examples):
    # Tokenize văn bản
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=13)
    
    # Mã hóa intent thành nhãn (labels) sử dụng LabelEncoder đã được fit
    tokenized["labels"] = label_encoder.transform(examples["intent"])  # Mã hóa nhãn thành chỉ số
    
    return tokenized

# Áp dụng mã hóa nhãn và tokenization cho cả train và test dataset
train_dataset = dataset["train"].map(encode_labels_and_tokenize, batched=True)
test_dataset = dataset["test"].map(encode_labels_and_tokenize, batched=True)

# Kiểm tra dữ liệu sau khi mã hóa nhãn và tokenization
  
print(train_dataset["labels"])  # Hiển thị mẫu đầu tiên trong train dataset

Generating train split: 311 examples [00:00, 22273.18 examples/s]
Generating test split: 311 examples [00:00, 34917.91 examples/s]


Danh sách nhãn gốc: ['clear_history' 'find_api' 'find_history' 'log_in' 'log_out']
Các nhãn đã mã hóa: [2 1 4 3 4 3 1 1 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


Map: 100%|██████████| 311/311 [00:00<00:00, 6965.35 examples/s]

[2, 1, 4, 3, 4, 3, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


# Finetuning

### Cấu hình tham số huấn luyện

In [132]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir='./logs',
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


### Finetuning

In [133]:
trainer.train()

c:\Users\qscvd\OneDrive\Documents\GitHub\makerthon\phoBERT-finetuning\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,1.381500
20,1.214100
30,0.840900
40,0.626600
50,0.525400
60,0.431800
70,0.366000
80,0.273600
90,0.239500
100,0.187900


TrainOutput(global_step=156, training_loss=0.4410160008149269, metrics={'train_runtime': 217.4125, 'train_samples_per_second': 5.722, 'train_steps_per_second': 0.718, 'total_flos': 8310833202984.0, 'train_loss': 0.4410160008149269, 'epoch': 4.0})

In [134]:
# Lưu mô hình và tokenizer của PhoBERT
tokenizer.save_pretrained("./results/final_model")
model.save_pretrained("./results/final_model")

# Test output

In [135]:
tokenizer = AutoTokenizer.from_pretrained("./results/final_model")
model = AutoModelForSequenceClassification.from_pretrained("./results/final_model")

In [142]:
# ['clear_history' 'find_api' 'find_history' 'log_in' 'log_out']

import torch

# Ví dụ văn bản muốn dự đoán
text = "đăng nhập đi"

# Tokenize văn bản
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Dự đoán với mô hình
with torch.no_grad():
    outputs = model(**inputs)

# Lấy kết quả dự đoán (logits)
logits = outputs.logits

# Lấy nhãn dự đoán (với argmax)
predicted_label = logits.argmax(dim=-1).item()

print(f"Predicted label: {predicted_label}")


Predicted label: 3
